In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# importing basic python libraries
import pandas as pd
import numpy as np

# importing data visualisation libraries
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import math
import seaborn as sns

# import scikit-learn split function
from sklearn.model_selection import train_test_split

# import K-Fold Cross Validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# import scikit-learn module to encode classes into one-hot coding matrix
from sklearn.preprocessing import LabelEncoder

# import keras module
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

Using TensorFlow backend.


## Python code to load the dataset into a Pandas dataframe:

In [3]:
# Specify what and where is the data file
filename = 'pima_diabetes.csv'

In [4]:
# Specify the fields with their names
col_names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']

In [5]:
# Load the data into a Pandas DataFrame
df = pd.read_csv(filename, names=col_names)

## Preprocess the dataset, find and clean missing values

In [6]:
# mark zero values as missing or NaN
df[['plas','pres','skin','test','mass','pedi','age']]=df[['plas','pres','skin','test','mass','pedi','age']].replace(0,np.NaN)

In [7]:
# checking percentage of unwanted data in dataset

def missing(df):
    print(df.isnull().sum() * 100/ len(df))

missing(df)

preg      0.000000
plas      0.651042
pres      4.557292
skin     29.557292
test     48.697917
mass      1.432292
pedi      0.000000
age       0.000000
class     0.000000
dtype: float64


In [8]:
# skin and test columns have considerable effect on data removal
# thus removing NaN values is not a good idea
# Instead replace the values with mean/median values

new_df=df.fillna(df.mean())
new_df.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148.0,72.0,35.00000,155.548223,33.6,0.627,50,1
1,1,85.0,66.0,29.00000,155.548223,26.6,0.351,31,0
2,8,183.0,64.0,29.15342,155.548223,23.3,0.672,32,1
3,1,89.0,66.0,23.00000,94.000000,28.1,0.167,21,0
4,0,137.0,40.0,35.00000,168.000000,43.1,2.288,33,1


# seperate dataset into input and output arrays

In [9]:
array = new_df.values
X = array[:,0:8]
Y = array[:,-1]

# split data into train ad test data

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

# Encode Class Values - One-Hot Coding

In [11]:
#encode class values as integers
encoder_train = LabelEncoder()
encoder_train.fit(Y_train)
encoded_Y_train = encoder_train.transform(Y_train)

#convert integers into one-hot coding format
onehot_Y_train = np_utils.to_categorical(encoded_Y_train)

#----------
#encode class values as integers
encoder_test = LabelEncoder()
encoder_test.fit(Y_test)
encoded_Y_test = encoder_test.transform(Y_test)

#convert integers into one-hot coding format
onehot_Y_test = np_utils.to_categorical(encoded_Y_test)

# MLP with Keras

In [12]:
# define a function to create a baseline model
# the network: 2-layered MLP = INPUT neurons + 1 hidden layer + OUTPUT layer
# optimization algorith: the adam
# loss function : cross entropy

def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(16, input_dim=8, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    
    #compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [13]:
# create the model
model = baseline_model()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                144       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 754
Trainable params: 754
Non-trainable params: 0
_________________________________________________________________


# train the model

In [15]:
# train the model
model.fit(X_train, onehot_Y_train, epochs=150, batch_size=10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/150
514/514 [==============================] - 1s 2ms/step - loss: 1.7626 - accuracy: 0.6576
Epoch 2/150
514/514 [==============================] - 0s 224us/step - loss: 1.0775 - accuracy: 0.6751
Epoch 3/150
514/514 [==============================] - 0s 225us/step - loss: 0.8191 - accuracy: 0.6829
Epoch 4/150
514/514 [==============================] - 0s 224us/step - loss: 0.7078 - accuracy: 0.6712
Epoch 5/150
514/514 [==============================] - 0s 225us/step - loss: 0.6963 - accuracy: 0.6595
Epoch 6/150
514/514 [==============================] - 0s 239us/step - loss: 0.6702 - accuracy: 0.6751
Epoch 7/150
514/514 [==============================] - 0s 235us/step - loss: 0.6281 - accuracy: 0.6848
Epoch 8/150
514/514 [==============================] - 0s 235us/step - loss: 0.7019 - accuracy: 0.6323
Epoch 9/150
514/514 [==============================] - 0s 236us/step - loss: 0.6942

## Evaluate model with K-Fold cross validation

In [16]:
# create kerasClassifier to evaluate the model
evaluator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5)

In [17]:
# evaluate the model
kfold = KFold(n_splits=10, shuffle = True, random_state=42)

In [18]:
# using k-fold cross validation to evaluate the model
results = cross_val_score(evaluator, X_test, onehot_Y_test, cv=kfold)

print("baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/200
228/228 [==============================] - 1s 4ms/step - loss: 2.2442 - accuracy: 0.5658
Epoch 2/200
228/228 [==============================] - 0s 741us/step - loss: 1.2709 - accuracy: 0.5439
Epoch 3/200
228/228 [==============================] - 0s 691us/step - loss: 1.0672 - accuracy: 0.6184
Epoch 4/200
228/228 [==============================] - 0s 675us/step - loss: 0.8585 - accuracy: 0.6228
Epoch 5/200
228/228 [==============================] - 0s 694us/step - loss: 0.9386 - accuracy: 0.6184
Epoch 6/200
228/228 [==============================] - 0s 688us/step - loss: 0.8863 - accuracy: 0.6842
Epoch 7/200
228/228 [==============================] - 0s 684us/step - loss: 0.8443 - accuracy: 0.6360
Epoch 8/200
228/228 [==============================] - 0s 737us/step - loss: 1.0242 - accuracy: 0.6053
Epoch 9/200
228/228 [==============================] - 0s 725us/step - loss: 0.9610 - accuracy: 0.5746
Epoch 10/200
228/228 [==============================] - 0s 691us/step - los

# Final Training and Validation Accuracy

In [19]:
#training accuracy
trainng_scores = model.evaluate(X_train, onehot_Y_train)
print("\n%s: %.2f%%" % (model.metrics_names[1], trainng_scores[1]*100))

514/514 [==============================] - 0s 550us/step

accuracy: 74.90%


In [20]:
#validation accuracy
test_scores = model.evaluate(X_test, onehot_Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], test_scores[1]*100))

254/254 [==============================] - 0s 46us/step

accuracy: 64.17%
